In [2]:
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from agent import *
from selection import *
from utils import *
from env import Environment
from simulation import constructAgent
from main import Config

In [8]:
MULTI_SELECTION_METHOD = 'LSTM-VAR'
config = {
    'reward': 3, 
    'sucker': 0, 
    'temptation': 5, 
    'punishment': 1, 
    'n_episodes': 2000, 
    'discount': 0.95,
    'play_epsilon': 1,
    'select_epsilon': 1,
    'epsilon_decay': 0.99,
    'min_epsilon': 0.01,
    'alpha': 0.1,
    'n_actions': 2,
    'h': 10,
    'state_repr': 'bi-repr',
    'batch_size': 64,
    'learning_rate': 1e-3,
}
config = Config(config)
env = Environment(config)
seed_everything()

In [9]:
agents = {}
index = 0
with HiddenPrints():
    # for _ in range(5):
        # agents[index] = constructAgent('TitForTat', config)
        # index += 1
    for _ in range(10):
        agents[index] = constructAgent('LSTMQN', config)
        index += 1
 
agents = lstm_variant_selection(config, agents, env)

for n in range(len(agents)):
    print('Agent{}: name:{}  final score:{}  play time:{}  times to play D:{}  ratio: {}'
        .format(n, agents[n].name, agents[n].running_score,
        len(agents[n].own_memory[:agents[n].play_times]), list(agents[n].own_memory[:agents[n].play_times]).count(1),
                list(agents[n].own_memory[:agents[n].play_times]).count(1)/len(agents[n].own_memory[:agents[n].play_times])))
print('The reward for total society: {}'.format(env.running_score/len(agents)))

100%|██████████| 2000/2000 [05:08<00:00,  6.49it/s]


Agent0: name:LSTMQN  final score:94.90004671163244  play time:2731  times to play D:2668  ratio: 0.9769315269132186
Agent1: name:LSTMQN  final score:94.53512405096731  play time:2184  times to play D:2132  ratio: 0.9761904761904762
Agent2: name:LSTMQN  final score:72.24022966612773  play time:3234  times to play D:3167  ratio: 0.979282622139765
Agent3: name:LSTMQN  final score:56.60100423693475  play time:2379  times to play D:107  ratio: 0.044976881042454814
Agent4: name:LSTMQN  final score:30.32478910200641  play time:8108  times to play D:240  ratio: 0.02960039467192896
Agent5: name:LSTMQN  final score:76.99412226035001  play time:2061  times to play D:2012  ratio: 0.9762251334303736
Agent6: name:LSTMQN  final score:62.46347902188574  play time:4845  times to play D:1363  ratio: 0.28132094943240454
Agent7: name:LSTMQN  final score:97.89910376212687  play time:2085  times to play D:2025  ratio: 0.9712230215827338
Agent8: name:LSTMQN  final score:48.315518361652174  play time:3602  ti

In [45]:
def get_index_from_action(action, idx):
    scale = lambda x: x+1 if x>=idx else x
    return list(map(scale, action))

for idx in agents:
    transitions = agents[idx].SelectMemory.memory
    _, actions, rewards, _ = zip(*transitions)
    actions = get_index_from_action(np.array(actions, dtype=int), idx)
    actions, rewards = np.array(actions), np.array(rewards)
    print(f'Agent {idx}:', end='')
    values, counts = np.unique(actions, return_counts=True)
    print(f' opponent_idx: {list(values)}, counts: {list(counts)} ', end='')
    dict_idx = {x: rewards[np.where(actions == x)] for x in values}
    print(f'rewards: {[np.mean(y) for _, y in dict_idx.items()]}')
    # print(f'rewards: {[np.std(y) for _, y in dict_idx.items()]}')
# plt.plot(actions)
# plt.show()
# print(rewards)

Agent 0: opponent_idx: [1, 3, 4, 8, 9], counts: [6, 2, 745, 146, 101] rewards: [0.8333333333333334, 1.0, 4.959731543624161, 4.972602739726027, 4.98019801980198]
Agent 1: opponent_idx: [0, 3, 4, 6, 8, 9], counts: [29, 1, 1, 62, 501, 406] rewards: [1.0, 1.0, 5.0, 1.5806451612903225, 4.972055888223553, 4.9655172413793105]
Agent 2: opponent_idx: [0, 4, 5, 6, 9], counts: [28, 1, 30, 1, 940] rewards: [1.0, 5.0, 1.0, 1.0, 4.965957446808511]
Agent 3: opponent_idx: [0, 1, 2, 4, 5, 7, 8, 9], counts: [1, 10, 33, 801, 2, 4, 148, 1] rewards: [1.0, 0.9, 0.8484848484848485, 2.9987515605493136, 1.0, 1.0, 3.0, 3.0]
Agent 4: opponent_idx: [0, 1, 2, 3, 5, 6, 8, 9], counts: [2, 2, 15, 8, 7, 303, 52, 611] rewards: [0.0, 0.0, 0.0, 3.0, 0.0, 2.597359735973597, 2.923076923076923, 2.988543371522095]
Agent 5: opponent_idx: [0, 4, 6, 8, 9], counts: [1, 1, 3, 18, 977] rewards: [1.0, 5.0, 1.0, 4.888888888888889, 4.981576253838281]
Agent 6: opponent_idx: [0, 1, 2, 3, 4, 7, 9], counts: [1, 13, 1, 331, 160, 1, 493] r